<a href="https://colab.research.google.com/github/AsTRIDgE/Machine_Learning/blob/master/Forest_Cover_Type_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Forest Cover Type - Kaggle Dataset

In [1]:
#Dataset from kaggle
#Classification Problem

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link = 'https://drive.google.com/open?id=1Jyi1mfRnh73qgpOYpiE1RwRDzDDZA_EU'

fluff, id = link.split('=')
print (id)

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('covtype.csv') 


     |████████████████████████████████| 993kB 3.6MB/s 

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

1Jyi1mfRnh73qgpOYpiE1RwRDzDDZA_EU


In [0]:
import pandas as pd
data = pd.read_csv('covtype.csv')

In [100]:
import numpy as np
#Using features with low corelation

corr = data.corr()
columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1,corr.shape[0]):
        if corr.iloc[i,j]>=0.5:
            if columns[j]:
                columns[j]=False
selected_columns = data.columns[columns]
data=data[selected_columns]
l = len(selected_columns) -1
print(l)


51


In [0]:
X=data.iloc[0:100000,0:l] #Selecting samples out of 5 lakhs+ samples
y=data.iloc[0:100000,l]

In [102]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
print(X_new.shape)

(100000, 40)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
from sklearn.model_selection import train_test_split
X_train , X_test, Y_train, Y_test = train_test_split(X_new,y,test_size=0.2, random_state=1, stratify=y)

In [104]:
#Normalising 
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()              
X_train_std = sc.fit_transform(X_train)         #standardized feature
X_test_std = sc.fit_transform(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [105]:
from sklearn import metrics

lsvc.fit(X_train_std , Y_train)
Y_predict = lsvc.predict(X_test_std)                        #predicting output for test case
print(metrics.f1_score(Y_test, Y_predict, average='micro'))

0.8098499999999998
